In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.3 MB/s eta 0:00:

In [ ]:
import torch.nn as nn
import torch
from transformers import AutoProcessor, FlavaModel
from datasets import load_dataset, concatenate_datasets
from torch import cuda
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from tqdm.notebook import tqdm, trange
import numpy as np
import cv2
import os
from torch.optim import AdamW
import json
import collections

In [ ]:
class FLAVAModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fusion_model = FlavaModel.from_pretrained("facebook/flava-full")
        # freeze layers in Flava
        for param in self.fusion_model.parameters():
            param.requires_grad = False
        self.hidden_layer_1 = nn.Linear(768, 512)
        self.hidden_layer_2 = nn.Linear(512, 512)
        self.output_layer = nn.Linear(512, 4)
        self.act = nn.ReLU()
        self.act_2 = nn.Sigmoid()

        
    def forward(self, input_ids , pixel_values, token_type_ids , attention_mask):
        multimodal_embeddings = self.fusion_model(input_ids, pixel_values, token_type_ids, attention_mask).multimodal_embeddings
        hl_1 = self.act(self.hidden_layer_1(multimodal_embeddings[:,0,:]))
        hl_2 = self.act(self.hidden_layer_2(hl_1))
        BBox = self.act_2(self.output_layer(hl_2))
        return BBox

In [ ]:
dataset50k = load_dataset('jxu9001/cs6301project50k')
# dataset100k = load_dataset('jxu9001/cs6301project100k')
# dataset180k = load_dataset('jxu9001/cs6301project180k')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/40000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/jxu9001___parquet/jxu9001--cs6301project50k-5a03d43d2e4dfad0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def resize224(image_width, image_height, x, y, w, h):
    origLeft, origTop, origRight, origBottom = x, y, w, h
    return x / image_width, y / image_height, w / image_width, h / image_height

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)
processor = AutoProcessor.from_pretrained("facebook/flava-full")

cuda


In [ ]:
class FLAVADataset(Dataset):
    def __init__(self, hf_dataset, split):
        self.split = hf_dataset[split]

    def __len__(self):
        return len(self.split)

    def __getitem__(self, idx):
        # get sample
        sample = self.split[idx]
        # process sample
        inputs = processor(
            text=sample['expression'], 
            images=sample['image'].resize((224, 224)).convert('RGB'), 
            return_tensors='pt', 
            padding='max_length', 
            max_length=55
        )
        # return processed sample
        processed_sample = {
            'input_ids' : inputs['input_ids'], 
            'pixel_values' : inputs['pixel_values'], 
            'token_type_ids' : inputs['token_type_ids'], 
            'attention_mask' : inputs['attention_mask'],
            'BBox': torch.tensor(resize224(sample['img_width'], sample['img_height'], sample['x'], sample['y'], sample['w'], sample['h']))
        }
        return processed_sample

In [ ]:
def average_iou(bboxes):
    """
    Calculates the average IoU across all bounding boxes in the input array.
    :param bboxes: An np array of shape (2, n, 4) where bboxes[0] holds n predicted bounding boxes
                   and bboxes[1] holds n ground truth bounding boxes. Each box is represented by
                   (x, y, w, h) where (x, y) is the top left corner of the bounding box and (w, h)
                   are the dimensions of the bounding box.
    :return: The average IoU across all bounding boxes.
    """
    # get the coordinates of the intersection rectangle
    x1 = np.maximum(bboxes[0][:, 0], bboxes[1][:, 0])
    y1 = np.maximum(bboxes[0][:, 1], bboxes[1][:, 1])
    x2 = np.minimum(bboxes[0][:, 0] + bboxes[0][:, 2], bboxes[1][:, 0] + bboxes[1][:, 2])
    y2 = np.minimum(bboxes[0][:, 1] + bboxes[0][:, 3], bboxes[1][:, 1] + bboxes[1][:, 3])

    # calculate the area of intersection rectangle
    intersection_area = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)

    # calculate the area of both the prediction and ground truth rectangles
    pred_area = bboxes[0][:, 2] * bboxes[0][:, 3]
    gt_area = bboxes[1][:, 2] * bboxes[1][:, 3]

    # calculate the union area by subtracting the intersection area from the sum of prediction and ground truth areas
    union_area = pred_area + gt_area - intersection_area

    # calculate the IoU
    iou = intersection_area / union_area

    # calculate the average IoU across all bounding boxes
    avg_iou = np.mean(iou)

    return avg_iou

In [ ]:
num_train_epochs = 20
batch_size = 64
learning_rate = 1.0e-4
weight_decay = 0.01
warmup_steps = 0

train_dataset = FLAVADataset(dataset50k, 'train')
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = FLAVADataset(dataset50k, 'test')
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

VLModel = FLAVAModel().to(device)
model_path = os.path.join(os.getcwd(), "drive", "My Drive", "cs6301", 'model_epoch=14-loss=4.3689-iou=0.2204.pth')
updated_params = torch.load(model_path)
model_state_dict = VLModel.state_dict()
model_state_dict.update(updated_params)
VLModel.load_state_dict(model_state_dict)
criterion = nn.SmoothL1Loss(reduction='sum')
optimizer = AdamW(VLModel.parameters(), lr=learning_rate, weight_decay=weight_decay)
optim_path = os.path.join(os.getcwd(), "drive", "My Drive", "cs6301", 'optim_epoch=14-loss=4.3689-iou=0.2204.pth')
optimizer.load_state_dict(torch.load(optim_path))

stats_path = os.path.join(os.getcwd(), "drive", "My Drive", "cs6301", "metrics.json")
f = open(stats_path)
metrics = json.load(f)
f.close()
# metrics = collections.defaultdict(list)

`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.


Some weights of the model checkpoint at facebook/flava-full were not used when initializing FlavaModel: ['image_codebook.blocks.group_4.group.block_2.res_path.path.conv_2.weight', 'image_codebook.blocks.group_1.group.block_2.res_path.path.conv_2.weight', 'image_codebook.blocks.group_2.group.block_2.res_path.path.conv_2.weight', 'image_codebook.blocks.group_1.group.block_1.res_path.path.conv_1.bias', 'mmm_text_head.transform.dense.weight', 'image_codebook.blocks.group_4.group.block_2.res_path.path.conv_1.weight', 'image_codebook.blocks.group_4.group.block_1.res_path.path.conv_4.bias', 'image_codebook.blocks.group_3.group.block_1.res_path.path.conv_3.weight', 'image_codebook.blocks.group_1.group.block_1.res_path.path.conv_4.bias', 'mim_head.transform.dense.bias', 'image_codebook.blocks.group_3.group.block_1.res_path.path.conv_3.bias', 'image_codebook.blocks.group_1.group.block_2.res_path.path.conv_1.weight', 'mlm_head.bias', 'itm_head.pooler.dense.bias', 'image_codebook.blocks.group_1.gr

In [ ]:
# for epoch_num in tqdm(range(1, num_train_epochs + 1)):
#     train_loss = 0
#     train_iou = 0
#     # train for one epoch
#     for batch in tqdm(train_dataloader, leave=False, desc='Training Batches'):        
#         VLModel.train()
#         b_input_ids = torch.squeeze(batch['input_ids']).to(device)
#         b_pixel_values = torch.squeeze(batch['pixel_values']).to(device)
#         b_token_type_ids = torch.squeeze(batch['token_type_ids']).to(device)
#         b_attention_mask = torch.squeeze(batch['attention_mask']).to(device)
#         b_BBox = batch['BBox']

#         BBox_Actual = b_BBox
#         BBox_Predicted = VLModel(input_ids=b_input_ids, pixel_values=b_pixel_values, token_type_ids=b_token_type_ids, attention_mask=b_attention_mask).cpu()
#         loss = criterion(BBox_Predicted, BBox_Actual)
#         train_loss += loss.item()
#         train_iou += average_iou(np.stack([BBox_Actual.detach().numpy(), BBox_Predicted.detach().numpy()]))
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#     # get training metrics
#     train_loss /= len(train_dataloader)
#     train_iou /= len(train_dataloader)
    

#     # test @ end of each epoch
#     with torch.no_grad():
#         test_loss = 0
#         test_iou = 0
#         for batch in tqdm(test_dataloader, leave=False, desc='Testing Batches'): 
#             VLModel.eval()
#             b_input_ids = torch.squeeze(batch['input_ids']).to(device)
#             b_pixel_values = torch.squeeze(batch['pixel_values']).to(device)
#             b_token_type_ids = torch.squeeze(batch['token_type_ids']).to(device)
#             b_attention_mask = torch.squeeze(batch['attention_mask']).to(device)
#             b_BBox = batch['BBox']

#             BBox_Actual = b_BBox
#             BBox_Predicted = VLModel(input_ids=b_input_ids, pixel_values=b_pixel_values, token_type_ids=b_token_type_ids, attention_mask=b_attention_mask).cpu()
#             loss = criterion(BBox_Predicted, BBox_Actual)
#             test_loss += loss.item()
#             test_iou += average_iou(np.stack([BBox_Actual.detach().numpy(), BBox_Predicted.detach().numpy()]))
#         # get testing statistics
#         test_loss /= len(test_dataloader)
#         test_iou /= len(test_dataloader)
        
#     # save the model, optimizer and train/test metrics every epoch
#     model_state = {}
#     state_dict = VLModel.state_dict()
#     for name, param in state_dict.items():
#         if param.requires_grad:
#             model_state[name] = param
#     optim_state = optimizer.state_dict()
#     model_pth = 'model_epoch={}-loss={:.4f}-iou={:.4f}.pth'.format(epoch_num, train_loss, train_iou)
#     optim_pth = 'optim_epoch={}-loss={:.4f}-iou={:.4f}.pth'.format(epoch_num, train_loss, train_iou)
#     checkpt_path = os.path.join(os.getcwd(), "drive", "My Drive", "cs6301")
#     torch.save(model_state, os.path.join(checkpt_path, model_pth))
#     torch.save(optim_state, os.path.join(checkpt_path, optim_pth))
#     metrics[epoch_num] = [train_loss, train_iou, test_loss, test_iou]
#     with open(os.path.join(checkpt_path, 'metrics.json'), 'w') as f:
#         json.dump(metrics, f)

#     # print metrics
#     print('EPOCH = {} | TRAIN: loss = {:.4f}, iou = {:.4f} | TEST: loss = {:.4f}, iou = {:.4f}'.format(epoch_num, train_loss, train_iou, test_loss, test_iou))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def resize_bbox(original_bbox, new_dims):
    """
    Resizes an original bbox to fit the dimensions of a new image.
    """
    x1, y1, w1, h1 = original_bbox
    w2, h2 = new_dims

    # calculate the scaling factors for width and height
    width_scale = w2 / w1
    height_scale = h2 / h1

    # calculate the new coordinates of the top left corner of the bbox
    x2 = x1 * width_scale
    y2 = y1 * height_scale

    # calculate the new dimensions of the bbox
    w2 = w1 * width_scale
    h2 = h1 * height_scale

    # return the resized bbox as a tuple
    return (x2, y2, w2, h2)

def plot_bounding_boxes(image, pred_bbox, gt_bbox):
    """
    Plot an image with a ground truth bounding box and a predicted bounding box.
    """
    # Compute the corners of the predicted bounding box
    pred_x, pred_y, pred_w, pred_h = pred_bbox
    pred_corners = np.array([[pred_x, pred_y], [pred_x + pred_w, pred_y], [pred_x + pred_w, pred_y + pred_h], [pred_x, pred_y + pred_h]])
    
    # Compute the corners of the ground truth bounding box
    gt_x, gt_y, gt_w, gt_h = gt_bbox
    gt_corners = np.array([[gt_x, gt_y], [gt_x + gt_w, gt_y], [gt_x + gt_w, gt_y + gt_h], [gt_x, gt_y + gt_h]])
    
    # Create a figure and axis object
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image)

    # Add the predicted bounding box to the plot
    pred_rect = patches.Polygon(pred_corners, linewidth=2, edgecolor='b', facecolor='none')
    ax.add_patch(pred_rect)

    # Add the ground truth bounding box to the plot
    gt_rect = patches.Polygon(gt_corners, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(gt_rect)
    
    # Show the plot
    plt.show()

import time
times = []
indices = np.random.choice(len(dataset50k['test']), 50, replace=False)
for i in indices:
    curr_sample = dataset50k['test'][int(i)]
    image = curr_sample['image']
    expression = curr_sample['expression']
    img_width = curr_sample['img_width']
    img_height = curr_sample['img_height']
    x = curr_sample['x']
    y = curr_sample['y']
    w = curr_sample['w']
    h = curr_sample['h']
    start = time.time()
    inputs = processor(text=[expression], images=image.convert('RGB'), return_tensors='pt', padding=True).to(device)
    pred_bbox = VLModel(**inputs)[0].detach().cpu().numpy() * 224
    pred_bbox = resize_bbox(pred_bbox, (w, h))
    times.append(time.time() - start)
    gt_bbox = np.array([x, y, w, h])
    # plot_bounding_boxes(image, pred_bbox, gt_bbox)
    # print(expression)
print(np.mean(times))
print(sum(times))

0.08915791988372802
4.457895994186401
